In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import plotly.express as px
from pathlib import Path
import json
import datetime
import pandas as pd
import numpy as np

In [ ]:
from autoguitar.tuning.dataset import get_dataset

entries = []

# path = "../data/tuning_data_selected/2025-01-27_18-19-39.jsonl"
# path = "../data/tuning_data_selected/2025-02-18_15-08-46-limit-100.jsonl"
# path = "../data/tuning_data_selected/2025-02-18_15-16-36-motor-wait.jsonl"
path = "../data/tuning_data_selected/2025-02-18_15-25-37-instant.jsonl"

with Path(path).open() as f:
    for line in f:
        data = json.loads(line)
        entry = {"kind": data["kind"], "added_at_network_timestamp": data["added_at_network_timestamp"]}

        if data["kind"]in ["model_based_tuner_strategy", "tuner"]:
            entry = {**entry, **data["value"]}
            entry["network_timestamp"] = pd.to_datetime(entry["network_timestamp"], unit="s")
        elif data["kind"] == "all_motors_status":
            motor_status = [x for x in data["value"]["status"] if x["motor_number"] == 0]
            assert len(motor_status) == 1
            motor_status = motor_status[0]

            entry["steps"] = motor_status["cur_steps"]
            entry["target_steps"] = motor_status["target_steps"]
            entry["network_timestamp"] = pd.to_datetime(data["value"]["network_timestamp"]) - pd.Timedelta(hours=1)

        entries.append(entry)

In [ ]:
df = pd.DataFrame(entries)
df = df.sort_values("network_timestamp").set_index("network_timestamp")
# df.index = pd.to_datetime(df.index, unit='s')
df = df.loc[df.index > datetime.datetime(2025, 1, 27, 17, 24, 0)]

In [ ]:
df

In [ ]:
df

In [ ]:
px.scatter(df, x=df.index, y=["intercept", "current_intercept", "instant_intercept"])

In [ ]:
import plotly.graph_objects as go
import plotly.subplots

df2 = df.loc[df["kind"] == "model_based_tuner_strategy"]

fig = plotly.subplots.make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=df2.index,
        y=df2["frequency"],
        mode="lines",
        name="frequency",
    )
)
fig.add_trace(
    go.Scatter(
        x=df2.index,
        y=df2["target_frequency"],
        mode="lines",
        name="target_frequency",
    )
)

# fig.add_trace(
#     go.Scatter(
#         x=df2.index,
#         y=df2["instant_intercept"],
#         # mode="lines",
#         # marker=dict(color="red"),
#         name="instant_intercept",
#     ),
#     secondary_y=True,
# )

fig.add_trace(
    go.Scatter(
        x=df2.index,
        y=df2["cur_steps"],
        # mode="lines",
        # marker=dict(color="red"),
        name="cur_steps",
    ),
    secondary_y=True,
)
# fig.add_trace(
#     go.Scatter(
#         x=df2.index,
#         y=df2["target_steps"],
#         # mode="lines",
#         # marker=dict(color="red"),
#         name="target_steps",
#     ),
#     secondary_y=True,
# )


fig.update_layout(
    xaxis=dict(title="Datetime"),
    title="Frequency over Time",
)

In [ ]:
px.scatter(df, x=df.index, y=["cur_steps", "estimated_target_steps"])

In [ ]:
df